# Optimization 2: Algorithms and Constraints

Florian Oswald
Sciences Po, 2021

## Bracketing

* A derivative-free method for *univariate* $f$
* works only on **unimodal** function $f$
* We can find a *bracket* in which the global minimum exists if we can find points $a < b < c$ such that

$$f(a) \geq f(b) < f(c) \text{ or } f(a) > f(b) \leq f(c)$$

![](unimodal.png)

## Initial bracket

* To start with, we need to identify first a region where bracketing will work.
* Once we found a valid interval, we shrink it until we find the optimimum.
* Here is an algorithm which will find a valid bracket, taking successive steps into a direction.

* The function `bracket_minimum` (next slide) in action:

![](bracketing.png)

In [ ]:
# algo 3.1 from Kochenderfer and Wheeler
function bracket_minimum(f, x=0; s=1e-2, k=2.0) 
    a, ya = x, f(x)
    b, yb = a + s, f(a + s) 
    if yb > ya
        a, b = b, a
        ya, yb = yb, ya s = -s
    end
    while true
    c, yc = b + s, f(b + s) 
        if yc > yb
            return a < c ? (a, c) : (c, a) 
        end
        a, ya, b, yb = b, yb, c, yc
        s *= k 
    end
end

## Fibonacci Search

* Suppose we have a bracket $[a,b]$ on $f$ and that we can only evaluate $f$ twice.
* Where to evaluate it?
* Let's split the interval into 3 equal pieces:

![](fibo1.png)


* If we move the points to the center, we can do better:

![](fibo2.png)


* for $n$ queries of $f$, we get the following sequence
* This can be determined analytically as $F_n = \frac{\psi^n - (1-\psi)^n}{\sqrt{5}}$
* and $\psi = (1+\sqrt{5})/2$ is called the *golden ratio*, which determines successive steps in the *Fibonacci Sequence*:

$$\frac{F_n}{F_{n-1}} = \psi \frac{1-s^{n+1}}{1-s^n}, \quad s=\frac{1-\sqrt{5}}{1+\sqrt{5}}\approx-0.382$$

![](fibo3.png)

* *Golden Section Search* uses the *golden ratio* to approximate the Fibonacci sequence.
* This trades off a minimal number of function evaluations with the best placement of evaluation points.

![](golden1.png)

![](golden2.png)

In [ ]:
using Plots
f(x) = exp(x) - x^4
plot(f,0,2, label = "f")

## Optim.jl

* Let's optimize this function with Optim!
* A typical call looks like this:

```julia
function your_function(x) ... end
using Optim
optimize(your_function,from,to,which_method_to_use)
```

In [ ]:
using Optim
minf(x) = -f(x)
brent = optimize(minf,0,2,Brent())
golden = optimize(minf,0,2,GoldenSection())
vline!([Optim.minimizer(brent)],label = "brent")
vline!([Optim.minimizer(golden)], label = "golden")

* Using Bracketing methods on non-unimodal functions is a *bad idea*
* We identify a different optimum depending on where we start to search.

In [ ]:
f2 = x->sin(x) + 0.1*abs(x)
x_arr = collect(range(3π/2-8.5, stop=3π/2+5.5, length=151))
y_arr = f2.(x_arr) 
plot(x_arr,y_arr,legend = false)

In [ ]:
ob = optimize(f2,1,5,Brent())
vline!([Optim.minimizer(ob)])

In [ ]:
ob = optimize(f2,-2.5,2.5,Brent())
vline!([Optim.minimizer(ob)])

In [ ]:
ob = optimize(f2,-2.5,10,Brent())
vline!([Optim.minimizer(ob)])

### Bisection Methods

* Root finding: `Roots.jl`
* Root finding in multivariate functions: [`IntervalRootFinding.jl`](https://github.com/JuliaIntervals/IntervalRootFinding.jl/)

In [ ]:
using Roots
# find the zeros of this function:
f(x) = exp(x) - x^3
plot(f,-2,5, label = "f")

In [ ]:
fzero(f, -2,4)   

In [ ]:
fzero(f,4,5)   

In [ ]:
using IntervalRootFinding, IntervalArithmetic

In [ ]:
rts = roots(f, -2..5, IntervalRootFinding.Bisection)

## Rosenbrock Banana and Optim.jl

* We can supply the objective function and - depending on the solution algorithm - the gradient and hessian as well.

In [ ]:
using Optim
using OptimTestProblems
for (name, prob) in MultivariateProblems.UnconstrainedProblems.examples
   println(name)
end

In [ ]:
prob = UnconstrainedProblems.examples["Rosenbrock"]
ro = prob.f
g! = prob.g!
h! = prob.h!

## Comparison Methods

* We will now look at a first class of algorithms, which are very simple, but sometimes a good starting point.
* They just *compare* function values.
* *Grid Search* : Compute the objective function at $G=\{x_1,\dots,x_N\}$ and pick the highest value of $f$. 
	* This is very slow.
	* It requires large $N$.
	* But it's robust (will find global optimizer for large enough $N$)

In [ ]:
# grid search on rosenbrock
grid = collect(-1.0:0.1:3);  # grid spacing is important!
grid2D = [[i;j] for i in grid,j in grid];
val2D = map(ro,grid2D);
r = findmin(val2D);
println("grid search results in minimizer = $(grid2D[r[2]])")

## Local Descent Methods

* Applicable to multivariate problems
* We are searching for a *local model* that provides some guidance in a certain region of $f$ over **where to go to next**.
* Gradient and Hessian are informative about this.

### Local Descent Outline

All descent methods follow more or less this structure. At iteration $k$,

1. Check if candidate $\mathbf{x}^{(k)}$ satisfies stopping criterion:
    * if yes: stop
    * if no: continue
2. Get the local *descent direction*  $\mathbf{d}^{(k)}$, using gradient, hessian, or both.
3. Set the *step size*, i.e. the length of the next step, $\alpha^k$
4. Get the next candidate via
    $$\mathbf{x}^{(k+1)} \longleftarrow \alpha^k\mathbf{d}^{(k)}$$

### The Line Search Strategy

* An algorithm from the line search class  chooses a direction $\mathbf{d}^{(k)} \in \mathbb{R}^n$ and searches along that direction starting from the current iterate $x_k \in \mathbb{R}^n$ for a new iterate $x_{k+1} \in \mathbb{R}^n$ with a lower function value.
* After deciding on a direction $\mathbf{d}^{(k)}$, one needs to decide the *step length* $\alpha$ to travel by solving
	$$ \min_{\alpha>0} f(x_k + \alpha \mathbf{d}^{(k)}) $$
* In practice, solving this exactly is too costly, so algos usually generate a sequence of trial values $\alpha$ and pick the one with the lowest $f$.

![](line-search.png)

In [ ]:
using LineSearches

algo_hz = Optim.Newton(linesearch = HagerZhang())    # Both Optim.jl and IntervalRootFinding.jl export `Newton`
res_hz = Optim.optimize(ro, g!, h!, prob.initial_x, method=algo_hz)

### The Trust Region Strategy

* First choose max step size, then the direction
* Finds the next step $\mathbf{x}^{(k+1)}$ by minimizing a model of $\hat{f}$ over a *trust region*, centered on $\mathbf{x}^{(k)}$
    * 2nd order Tayloer approx of $f$ is common.
* Radius $\delta$ of trust region is changed based on how well $\hat{f}$ fits $f$ in trust region.
* Get $\mathbf{x'}$ via
    $$
    \begin{aligned}
    \min_{\mathbf{x'}} &\quad \hat{f}(\mathbf{x'}) \\
    \text{subject to } &\quad \Vert \mathbf{x}-\mathbf{x'} \leq \delta \Vert
    \end{aligned}
    $$

![](trust-region.png)

In [ ]:
# Optim.jl has a TrustRegion for Newton (see below for Newton's Method)
NewtonTrustRegion(; initial_delta = 1.0, # The starting trust region radius
                    delta_hat = 100.0, # The largest allowable trust region radius
                    eta = 0.1, #When rho is at least eta, accept the step.
                    rho_lower = 0.25, # When rho is less than rho_lower, shrink the trust region.
                    rho_upper = 0.75) # When rho is greater than rho_upper, grow the trust region (though no greater than delta_hat).
res = Optim.optimize(ro, g!, h!, prob.initial_x, method=NewtonTrustRegion())

![](trust-region2.png)

### Stopping criteria

1. maximum number of iterations reached
2. absolute improvement $|f(x) - f(x')| \leq \epsilon$
3. relative improvement $|f(x) - f(x')| / |f(x)| \leq \epsilon$
4. Gradient close to zero $|g(x)| \approx 0$


## Gradient Descent

The steepest descent is *opposite the gradient*.
* Here we define
    $$\mathbf{g}^{(k)} = \nabla f(\mathbf{d}^{(k)})$$
* And our descent becomes
    $$\mathbf{d}^{(k)} = -\nabla \frac{\mathbf{g}^{(k)} }{\Vert\mathbf{g}^{(k)}\Vert }$$
* Minimizing wrt step size results in a jagged path (each direction is orthogonal to previous direction!) See Kochenderfer and Wheeler chapter 5, equation (5.6)
    $$\alpha^{(k)} = \arg \min{\alpha} f(\mathbf{x}^{(k)} + \alpha \mathbf{d}^{(k)}) $$

In [ ]:
# Optim.jl again: Default constructor for a GradientDescent solver
GradientDescent(; alphaguess = LineSearches.InitialPrevious(),
                  linesearch = LineSearches.HagerZhang(),
                  P = nothing,
                  precondprep = (P, x) -> nothing)

In [ ]:
using Interact

In [ ]:
# Those cool graphs are modified from Patrick Kofod Mogensens' Julia Con presentation
# https://github.com/pkofod/JC2017
# some are identical copies.
x = [-4,3.]
res = optimize(ro, x, GradientDescent(), Optim.Options(store_trace=true, extended_trace=true))
@manipulate for ix = slider(1:Optim.iterations(res), value = 1)
    contour(-4.1:0.01:2, -1.5:0.01:4.1, (x,y)->sqrt(ro([x, y])), fill=true, color=:deep, legend=false)
    xtracemat = hcat(Optim.x_trace(res)...)
    plot!(xtracemat[1, 1:ix], xtracemat[2, 1:ix], mc = :white, lab="")
    scatter!(xtracemat[1:1,ix], xtracemat[2:2,ix], mc=:black, msc=:red, lab="")
    scatter!([1.], [1.], mc=:red, msc=:red, lab="")
    scatter!(x[1:1], x[2:2], mc=:yellow, msc=:black, label="start", legend=true)
end

In [ ]:
# zooming in: along the valley, it moves really slow!
x = [-1,1.]
res = optimize(ro, x, GradientDescent(), Optim.Options(store_trace=true, extended_trace=true))
@manipulate for ix = slider(1:Optim.iterations(res), value = 1)
    contour(-2.5:0.01:2, -1.5:0.01:2, (x,y)->sqrt(ro([x, y])), fill=true, color=:deep, legend=false)
    xtracemat = hcat(Optim.x_trace(res)...)
    plot!(xtracemat[1, 1:ix], xtracemat[2, 1:ix], mc = :white, lab="")
    scatter!(xtracemat[1:1,ix], xtracemat[2:2,ix], mc=:black, msc=:red, lab="")
    scatter!([1.], [1.], mc=:red, msc=:red, lab="")
    scatter!(x[1:1], x[2:2], mc=:yellow, msc=:black, label="start", legend=true)
end

* pretty zig-zaggy
* it may not be a huge issue for final convergence, but we take a lot of *unncessary* steps back and forth.
* so in terms of computational cost, we incur quite a lot here.
* zoom in even more:

In [ ]:
x = [-4, -3.]
@manipulate for ix = slider(120:160, value = 120)
    c = contour(-.85:0.01:-0.60, 0:0.01:1.2, (x,y)->sqrt(ro([x, y])), fill=true, color=:deep, legend=false)
    res = optimize(ro, x, GradientDescent(), Optim.Options(store_trace=true, extended_trace=true))
    xtracemat = hcat(Optim.x_trace(res)...)
    
    plot!(xtracemat[1, 120:ix], xtracemat[2, 120:ix], mc = :white, legend=true, label="Gradient Descent")
    scatter!(xtracemat[1:1,ix], xtracemat[2:2,ix], mc=:black, msc=:red, label = "")
    scatter!(xtracemat[1:1,120], xtracemat[2:2,120], mc=:blue, msc=:blue, label="start")
    p = plot(1:ix, [Optim.trace(res)[i].metadata["Current step size"] for i  = 1:ix], label = "step size")
    plot(c, p, layout=(2,1))
end

## Conjugate Gradient Descent

* Tries to avoid the jagged path problem
* It combines several methods: it minimizes a quadratic form of $f$
* The next descent direction uses the gradient *plus* some additional info from the previous step.

In [ ]:
x = [-4,3.]
res = optimize(ro, x, ConjugateGradient(), Optim.Options(store_trace=true, extended_trace=true))
@manipulate for ix = slider(1:Optim.iterations(res), value = 1)
    contour(-4.1:0.01:2, -1.5:0.01:4.1, (x,y)->sqrt(ro([x, y])), fill=true, color=:deep, legend=false)
    xtracemat = hcat(Optim.x_trace(res)...)
    plot!(xtracemat[1, 1:ix], xtracemat[2, 1:ix], mc = :white, lab="")
    scatter!(xtracemat[1:1,ix], xtracemat[2:2,ix], mc=:black, msc=:red, lab="")
    scatter!([1.], [1.], mc=:red, msc=:red, lab="")
    scatter!(x[1:1], x[2:2], mc=:yellow, msc=:black, label="start", legend=true)
end

In [ ]:
x0 = [-4, -3.]
i0 = 15
res = optimize(ro, x0, ConjugateGradient(), Optim.Options(store_trace=true, extended_trace=true))

@manipulate for ix = slider(i0:Optim.iterations(res), value = i0)
    c = contour(0.5:0.01:1.1, 0:0.01:1.1, (x,y)->sqrt(ro([x, y])), fill=true, color=:deep, legend=false)
    xtracemat = hcat(Optim.x_trace(res)...)
    
    plot!(xtracemat[1, i0:ix], xtracemat[2, i0:ix], mc = :white, legend=:bottomright, label="Gradient Descent")
    scatter!(xtracemat[1:1,ix], xtracemat[2:2,ix], mc=:black, msc=:red, label = "")
    scatter!(xtracemat[1:1,i0], xtracemat[2:2,i0], mc=:blue, msc=:blue, label="start")
    scatter!([1.], [1.], mc=:red, msc=:red, lab="")

    p = plot(1:ix, [Optim.trace(res)[i].metadata["Current step size"] for i  = 1:ix], label = "step size")
    plot(c, p, layout=(2,1))
end

## Second Order Methods

### Newton's Method

* We start with a 2nd order Taylor approx over x at step $k$:
    $$q(x) = f(x^{(k)}) + (x - x^{(k)}) f'(x^{(k)}) + \frac{(x - x^{(k)})^2}{2}f''(x^{(k)})$$
* We form first order conditions (set it's root equal to zero) and rearrange to find the next step $k+1$:
    $$
    \begin{aligned}
    \frac{\partial q(x)}{\partial x} &= f'(x^{(k)}) + (x - x^{(k)}) f''(x^{(k)}) = 0 \\
    x^{(k+1)} &= x^{(k)} - \frac{f'(x^{(k)})}{f''(x^{(k)})}
    \end{aligned}
    $$

* The same argument works for multidimensional functions by using Hessian and Gradient
* We would get a descent $\mathbf{d}^k$ by setting:
    $$\mathbf{d}^k = -\frac{\mathbf{g}^{k}}{\mathbf{H}^{k}}$$
* There are several options to avoid (often costly) computation of the Hessian $\mathbf{H}$:
1. Quasi-Newton updates $\mathbf{H}$ starting from identity matrix
2. Broyden-Fletcher-Goldfarb-Shanno (BFGS) does better with approx linesearch
3. L-BFGS is the limited memory version for large problems

In [ ]:
x = [-1., -1.]
res = optimize(ro, x, GradientDescent(), Optim.Options(store_trace=true, extended_trace=true))
res2 = optimize(ro, g!, h!, x, Optim.Newton(), Optim.Options(store_trace=true, extended_trace=true))
@manipulate for ix = slider(1:Optim.iterations(res2), value = 1)
    p = contour(-2.5:0.01:2, -1.5:0.01:2, (x,y)->sqrt(ro([x, y])), fill=true, color=:deep, legend=false)
    xtracemat = hcat(Optim.x_trace(res)...)
    plot!(xtracemat[1, 1:ix], xtracemat[2, 1:ix], mc = :white,  label="Gradient Descent", legend=true)
    scatter!(xtracemat[1:1,ix], xtracemat[2:2,ix], mc=:black, msc=:red, label="")
    
    xtracemat2 = hcat(Optim.x_trace(res2)...)
    plot!(xtracemat2[1, 1:ix], xtracemat2[2, 1:ix], c=:blue, label="Newton")
    scatter!(xtracemat2[1:1,ix], xtracemat2[2:2,ix], mc=:black, msc=:blue, label="")
    scatter!([1.], [1.], mc=:red, msc=:red, label="")
    scatter!(x[1:1], x[2:2], mc=:yellow, msc=:yellow, label="start")
end

In [ ]:
x = [-1., -1.]
res = optimize(ro, x, ConjugateGradient(), Optim.Options(store_trace=true, extended_trace=true))
res2 = optimize(ro, g!, h!, x, Optim.Newton(), Optim.Options(store_trace=true, extended_trace=true))
@manipulate for ix = slider(1:Optim.iterations(res2), value = 1)
    p = contour(-2.5:0.01:2, -1.5:0.01:2, (x,y)->sqrt(ro([x, y])), fill=true, color=:deep, legend=false)
    xtracemat = hcat(Optim.x_trace(res)...)
    plot!(xtracemat[1, 1:ix], xtracemat[2, 1:ix], mc = :white,  label="Conjugate Gradient", legend=true)
    scatter!(xtracemat[1:1,ix], xtracemat[2:2,ix], mc=:black, msc=:red, label="")
    
    xtracemat2 = hcat(Optim.x_trace(res2)...)
    plot!(xtracemat2[1, 1:ix], xtracemat2[2, 1:ix], c=:blue, label="Newton")
    scatter!(xtracemat2[1:1,ix], xtracemat2[2:2,ix], mc=:black, msc=:blue, label="")
    scatter!([1.], [1.], mc=:red, msc=:red, label="")
    scatter!(x[1:1], x[2:2], mc=:yellow, msc=:yellow, label="start")
end

In [ ]:
@time optimize(ro, [0.0, 0.0], Optim.Newton(),Optim.Options(show_trace=false))

In [ ]:
@time optimize(ro, g!, h!, [0.0, 0.0], Optim.Newton(),Optim.Options(show_trace=false))

In [ ]:
@time optimize(ro, g!, h!,  [-1.0, 3.0], BFGS());

In [ ]:
# low memory BFGS
@time optimize(ro, g!, h!,  [0.0, 0.0], LBFGS());

## Direct Methods

* No derivative information is used - *derivative free*
* If it's very hard / impossible to provide gradient information, this is our only chance.
* Direct methods use other criteria than the gradient to inform the next step (and ulimtately convergence).

### Cyclic Coordinate Descent -- Taxicab search

* We do a line search over each dimension, one after the other
* *taxicab* because the path looks like a NYC taxi changing direction at each block.
* given $\mathbf{x}^{(1)}$, we proceed
    $$
    \begin{aligned}
    \mathbf{x}^{(2)} &= \arg \min_{x_1} f(x_1,x_2^{(1)},\dots,x_n^{(1)}) \\
    \mathbf{x}^{(3)} &= \arg \min_{x_2} f(x_1^{(2)},x_2,x_3^{(2)}\dots,x_n^{(2)}) \\    
    \end{aligned}
    $$
* unfortunately this can easily get stuck because it can only move in 2 directions.

In [ ]:
# start to setup a basis function, i.e. unit vectors to index each direction:
basis(i, n) = [k == i ? 1.0 : 0.0 for k in 1 : n]
function cyclic_coordinate_descent(f, x, ε) 
    Δ, n = Inf, length(x)
    while abs(Δ) > ε
        x′ = copy(x) 
            for i in 1 : n
                d = basis(i, n)
                x = line_search(f, x, d) 
            end
        Δ = norm(x - x′) 
    end
    return x 
end  

### General Pattern Search

* We search according to an arbitrary *pattern* $\mathcal{P}$ of candidate points, anchored at current guess $\mathbf{x}$.
* With step size $\alpha$ and set $\mathcal{D}$ of directions
    $$ \mathcal{P} = {\mathbf{x} + \alpha \mathbf{d} \text{ for } \mathbf{d}\in\mathcal{D} }$$
* Convergence is guaranteed under conditions:
    * $\mathcal{D}$ must be a positive spanning set: at least one $\mathbf{d}\in\mathcal{D}$ has a non-zero gradient.

![](spanning-set.png)

In [ ]:
function generalized_pattern_search(f, x, α, D, ε, γ=0.5) 
    y, n = f(x), length(x)
    evals = 0
    while α > ε
        improved = false
        for (i,d) in enumerate(D)
            x′ = x + α*d 
            y′ = f(x′) 
            evals += 1
            if y′ < y
                x, y, improved = x′, y′, true
                D = pushfirst!(deleteat!(D, i), d) 
                break
            end 
        end
        if !improved 
            α *= γ
        end 
    end
    println("$evals evaluations")
    return x 
end

In [ ]:
D = [[1,0],[0,1],[-1,-0.5]]
y=generalized_pattern_search(ro,zeros(2),0.8,D,1e-6 )

## Bracketing for Multidimensional Problems: Nelder-Mead

* The Goal here is to find the simplex containing the local minimizer $x^*$
* In the case where $f$ is n-D, this simplex has $n+1$ vertices
* In the case where $f$ is 2-D, this simplex has $2+1$ vertices, i.e. it's a triangle.
* The method proceeds by evaluating the function at all $n+1$ vertices, and by replacing the worst function value with a new guess.
* this can be achieved by a sequence of moves:
	* reflect
	* expand
	* contract
	* shrink
	movements.



![](nelder-mead.png)


* this is a very popular method. The matlab functions `fmincon` and `fminsearch` implements it.
* When it works, it works quite fast.
* No derivatives required.

In [ ]:
@time nm=optimize(ro, [0.0, 0.0], NelderMead());
nm.minimizer

* But.


## Bracketing for Multidimensional Problems: Comment on Nelder-Mead

> Lagarias et al. (SIOPT, 1999):
At present there is no function in any dimension greater than one, for which the original Nelder-Mead algorithm has been proved to converge to a minimizer.

>Given all the known inefficiencies and failures of the Nelder-Mead algorithm [...], one might wonder why it is used at all, let alone why it is so extraordinarily popular.




## things to read up on

* Divided Rectangles (DIRECT)
* simulated annealing and other stochastic gradient methods



## Stochastic Optimization Methods

* Gradient based methods like steepest descent may be susceptible to getting stuck at local minima.
* Randomly shocking the value of the descent direction may be a solution to this.
* For example, one could modify our gradient descent from before to become


 $$\mathbf{x}^{(k+1)} \longleftarrow \mathbf{x}^{(k)} +\alpha^k\mathbf{g}^{(k)} + \mathbf{\varepsilon}^{(k)}$$

* where $\mathbf{\varepsilon}^{(k)} \sim N(0,\sigma_k^2)$, decreasing with $k$.
* This *stochastic gradient descent* is often used when training neural networks.


### Simulated Annealing

* We specify a *temperature* that controls the degree of randomness.
* At first the temperature is high, letting the search jump around widely. This is to escape local minima.
* The temperature is gradually decreased, reducing the step sizes. This is to find the local optimimum in the *best* region.
* At every iteration $k$, we accept new point $\mathbf{x'}$ with

$$
\Pr(\text{accept }\mathbf{x'}) = \begin{cases}
1 & \text{if }\Delta y \leq0 \\
\min(e^{\Delta y / t},1) & \text{if }\Delta y > 0 
\end{cases}
$$

* here $\Delta y = f(\mathbf{x'}) - f(\mathbf{x})$, and $t$ is the *temperature*.
* $\Pr(\text{accept }\mathbf{x'})$ is called the **Metropolis Criterion**, building block of *Accept/Reject* algorithms.

In [ ]:
# f: function
# x: initial point
# T: transition distribution
# t: temp schedule, k_max: max iterations
function simulated_annealing(f, x, T, t, k_max) 
    y = f(x)
    ytrace = zeros(typeof(y),k_max)
    x_best, y_best = x, y 
    for k in 1 : k_max
        x′ = x + rand(T)
        y′ = f(x′)
        Δy = y′ - y
        if Δy ≤ 0 || rand() < exp(-Δy/t(k))
            x, y = x′, y′ 
        end
        if y′ < y_best
            x_best, y_best = x′, y′
        end 
        ytrace[k] = y_best
    end
    return x_best,ytrace
end

In [ ]:
function ackley(x, a=20, b=0.2, c=2π) 
    d = length(x)
    return -a*exp(-b*sqrt(sum(x.^2)/d)) - exp(sum(cos.(c*xi) for xi in x)/d) + a + exp(1)
end
surface(-30:0.1:30,-30:0.1:30,(x,y)->ackley([x, y]),cbar=false)

In [ ]:
# first with optim!
res = Optim.optimize(ackley, [-30.0,-30], [30.0,30.0], [-20.0,-20], SAMIN(), Optim.Options(iterations=10^6))

In [ ]:
SAMIN(; nt::Int = 5     # reduce temperature every nt*ns*dim(x_init) evaluations
        ns::Int = 5     # adjust bounds every ns*dim(x_init) evaluations
        rt::T = 0.9     # geometric temperature reduction factor: when temp changes, new temp is t=rt*t
        neps::Int = 5   # number of previous best values the final result is compared to
        f_tol::T = 1e-12 # the required tolerance level for function value comparisons
        x_tol::T = 1e-6 # the required tolerance level for x
        coverage_ok::Bool = false # if false, increase temperature until initial parameter space is covered
        verbosity::Int = 0) # scalar: 0, 1, 2 or 3 (default = 0).


In [ ]:
res = Optim.optimize(ackley, [-30.0,-30], [30.0,30.0], [-20.0,-20], SAMIN(nt = 15, rt = 0.4), Optim.Options(iterations=10^6))

In [ ]:
# now with our hand-rolled version!
p = Any[]
using Distributions
niters = 10000
temps = (1,10,50)
sigs = (1,5,30)
push!(p,[plot(x->i/x,1:niters,title = "tmp $i",lw=2,ylims = (0,1),leg = false) for i in temps]...)
for sig in sigs, t1 in temps
    y = simulated_annealing(ackley,[15,15],MvNormal(2,sig),x->t1/x,niters)[2][:]
    push!(p,plot(y,title = "sig = $sig",leg=false,lw=1.5,color="red",ylims = (0,20)))
end

In [ ]:
plot(p...,layout = (4,3))